In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#modelling
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

## import csv  data

In [4]:
df=pd.read_csv("/Users/kuldeepsharma/Downloads/stud.csv")

In [5]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
x=df.drop(columns=["math_score"],axis=1)

In [7]:
y=df['math_score']

In [10]:
#create column transformer with 3 types of transformer
cat_features=x.select_dtypes(include="object").columns
num_features=x.select_dtypes(exclude="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer=StandardScaler()
oh_transformer=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
        ("onehotencoder",oh_transformer,cat_features),
        ('tandardscaler',num_transformer,num_features)
    ]
)

In [13]:
x=preprocessor.fit_transform(x)

In [14]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [15]:
##separate dataset into train test split 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((800, 19), (200, 19))

# evaluate function to give all metrics after model training

In [18]:

def evaluate_model(true,prediction):
    mae=mean_absolute_error(true,prediction)
    mse=mean_squared_error(true,prediction)
    rmse=np.sqrt(mean_squared_error(true,prediction))
    r2_square=r2_score(true,prediction)
    return r2_square,mae,rmse

In [21]:
models={
    'linear regression':LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'decision tree':DecisionTreeRegressor(),
    'Random forest':RandomForestRegressor(),
    'XGBRegressor':XGBRegressor(),
    'cat boosting regressor':CatBoostRegressor(verbose=False),
    'AdaBoostRegressor':AdaBoostRegressor()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)#train model

    #make prediction
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    #evaluate train and test dataset
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_rmse,model_test_mae,model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("model performace for training set")
    print("- root mean squared error:{:.4f}".format(model_train_rmse))
    print("-  mean absolute error:{:.4f}".format(model_train_mae))
    print("- r2 score:{:.4f}".format(model_train_r2))
    print("------------------------------")
    print("model performace for test set".format(model_test_rmse))
    print("- root mean squared error:{:.4f}".format(model_test_rmse))
    print("-  mean absolute error:{:.4f}".format(model_test_mae))
    print("- r2 score:{:.4f}".format(model_train_r2))   
    r2_list.append(model_test_r2)
    print('='*35)
    print('\n')

linear regression
model performace for training set
- root mean squared error:4.2667
-  mean absolute error:0.8743
- r2 score:5.3231
------------------------------
model performace for test set
- root mean squared error:0.8804
-  mean absolute error:4.2148
- r2 score:5.3231


ridge
model performace for training set
- root mean squared error:4.2650
-  mean absolute error:0.8743
- r2 score:5.3233
------------------------------
model performace for test set
- root mean squared error:0.8806
-  mean absolute error:4.2111
- r2 score:5.3233


lasso
model performace for training set
- root mean squared error:5.2063
-  mean absolute error:0.8071
- r2 score:6.5938
------------------------------
model performace for test set
- root mean squared error:0.8253
-  mean absolute error:5.1579
- r2 score:6.5938


decision tree
model performace for training set
- root mean squared error:0.0187
-  mean absolute error:0.9997
- r2 score:0.2795
------------------------------
model performace for test set
- r